In [1]:
import pandas as pd

## Jigsaw Toxic Comments

In [2]:
jigsaw_train = pd.read_csv("./data/raw/jigsaw/train.csv")
jigsaw_test = pd.read_csv("./data/raw/jigsaw/test.csv")
jigsaw_test_labels = pd.read_csv("./data/raw/jigsaw/test_labels.csv")

In [3]:
jigsaw_test = pd.merge(jigsaw_test, jigsaw_test_labels, on="id", how="inner", validate="1:1")

In [4]:
jigsaw_df = pd.concat([jigsaw_train, jigsaw_test]).reset_index(drop=True)
jigsaw_normal = jigsaw_df.loc[(jigsaw_df.iloc[:, -6:] == 0).all(axis=1)]
jigsaw_normal_prompts = jigsaw_normal["comment_text"].tolist()

## Anthropic HH-RLHF

In [5]:
anthropic_train_helpful = pd.read_json("./data/raw/anthropic/helpful_base/train.jsonl", lines=True)
anthropic_test_helpful = pd.read_json("./data/raw/anthropic/helpful_base/test.jsonl", lines=True)

anthropic_train_helpful_online = pd.read_json("./data/raw/anthropic/helpful_online/train.jsonl", lines=True)
anthropic_test_helpful_online = pd.read_json("./data/raw/anthropic/helpful_online/test.jsonl", lines=True)

In [6]:
anthropic_df = pd.concat([anthropic_train_helpful, anthropic_test_helpful, anthropic_train_helpful_online, anthropic_test_helpful_online]).reset_index(drop=True)
anthropic_df["chosen"] = anthropic_df["chosen"].apply(lambda x: " ".join(x.split("\n\n")[1].split()[1:]))
anthropic_df = anthropic_df.drop_duplicates(subset=["chosen"]).reset_index(drop=True)

In [7]:
anthropic_normal_prompts = anthropic_df["chosen"].tolist()

In [8]:
normal_prompts = pd.DataFrame({"prompt": jigsaw_normal_prompts + anthropic_normal_prompts, "category": ["jigsaw"] * len(jigsaw_normal_prompts) + ["anthropic"] * len(anthropic_normal_prompts)})
normal_prompts = normal_prompts.sample(frac=1).reset_index(drop=True)
normal_prompts.to_csv("./data/processed/normal_prompts.csv", index=False)

In [9]:
normal_prompts

,prompt,category
0,REDIRECT Talk:1999 South Asian Football Federa...,jigsaw
1,Labour? \n\nIt's Labor.,jigsaw
2,So do you make a habit of spending all your ti...,jigsaw
3,"""\n\nThe World Would Be a Better Place Without...",jigsaw
4,==WP:PINQ== \n Hi! i think you could be intere...,jigsaw
...,...,...
226021,They were certainly sufficient for the majorit...,jigsaw
226022,"January 2011 (UTC)\n\nThanks, this is exactly ...",jigsaw
226023,or?: \nSimilar arguments have been made that l...,jigsaw
226024,hi buddy \n\nYou say that my page is blatant a...,jigsaw
